In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [3]:
adata = sc.read_h5ad('../datasets/slidetag_LN/slidetag_LN.h5ad')
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 5778 × 4000
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types'
    uns: 'cell_type_colors', 'log1p'
    obsm: 'spatial'

In [6]:
adata.X = adata.X.toarray()

In [ ]:
from model import SpiceMix
from sklearn.neighbors import kneighbors_graph
import torch
K, num_pcs, n_neighbors, res_lo, res_hi = 20, 20, 10, .5, 2.
context = dict(device='cuda:2', dtype=torch.float64)
repli_list = ['slidetag']

A = kneighbors_graph(adata.obsm['spatial'],n_neighbors = 10)
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    print(i)
    obj = SpiceMix(
    K=20,
    lambda_Sigma_x_inv=1e-6, power_Sigma_x_inv=2,
    repli_list=repli_list,
    context=context,
    context_Y=context,
    )
    obj.Ys = [adata.X]
    obj.Gs = [2000]
    obj.GG = 2000
    obj.Ys = [G / obj.GG * K * Y / Y.sum(1).mean() for Y, G in zip(obj.Ys, obj.Gs)]
    obj.Ys = [torch.tensor(Y, **context).pin_memory() 
               if context['device'] != 'cpu' and context['device'] == 'cpu' else
               torch.tensor(Y, **context) for Y in obj.Ys]
    obj.Es = [np.vstack((A.tocoo().row, A.tocoo().col)).T]
    obj.Es_isempty = [True, True, True]

    obj.initialize(method='kmeans',random_state=i)
    for iiter in range(10):
        obj.estimate_weights(iiter=iiter, use_spatial=[False]*obj.num_repli)
        obj.estimate_parameters(iiter=iiter, use_spatial=[False]*obj.num_repli)
    obj.initialize_Sigma_x_inv()
    for iiter in range(1, 101):
        obj.estimate_parameters(iiter=iiter, use_spatial=[True]*obj.num_repli)
        obj.estimate_weights(iiter=iiter, use_spatial=[True]*obj.num_repli)
    adata.obsm['spicemix'+str(i)] = torch.cat(obj.Xs).cpu().numpy()
    np.save('../Benchmarking_results/slidetag/'+'spicemix'+str(i)+'.npy',adata.obsm['spicemix'+str(i)])

Global seed set to 0


0


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-03-24 06:22:31:INFO:Updating latent states
2024-03-24 06:22:32:INFO:Updating model parameters
2024-03-24 06:22:34:INFO:Q value = nan
2024-03-24 06:22:34:INFO:Updating latent states
2024-03-24 06:22:35:INFO:Updating model parameters
2024-03-24 06:22:36:INFO:Q value = nan
2024-03-24 06:22:36:INFO:Updating latent states
2024-03-24 06:22:37:INFO:Updating model parameters
2024-03-24 06:22:38:INFO:Q value = nan
2024-03-24 06:22:38:INFO:Updating latent states
2024-03-24 06:22:38:INFO:Updating model parameters
2024-03-24 06:22:39:INFO:Q value = nan
2024-03-24 06:22:39:INFO:Updating latent states
2024-03-24 06:22:39:INFO:Updating model parameters
2024-03-24 06:22:41:INFO:Q value = nan
2024-03-24 06:22:41:INFO:Updating latent states
2024-03-24 06:22:41:INFO:Updat

2024-03-24 06:29:10:INFO:Q value = -38071770.78898596
2024-03-24 06:29:10:INFO:Updating latent states
2024-03-24 06:29:10:INFO:Updating model parameters
2024-03-24 06:29:15:INFO:Q value = -38071862.15572468
2024-03-24 06:29:15:INFO:Updating latent states
2024-03-24 06:29:15:INFO:Updating model parameters
2024-03-24 06:29:22:INFO:Q value = -38071909.25013022
2024-03-24 06:29:22:INFO:Updating latent states
2024-03-24 06:29:22:INFO:Updating model parameters
2024-03-24 06:29:32:INFO:Q value = -38071666.13093613
2024-03-24 06:29:32:INFO:Updating latent states
2024-03-24 06:29:32:INFO:Updating model parameters
2024-03-24 06:29:46:INFO:Q value = -38071159.015224725
2024-03-24 06:29:46:INFO:Updating latent states
2024-03-24 06:29:46:INFO:Updating model parameters
2024-03-24 06:29:54:INFO:Q value = -38071119.93858827
2024-03-24 06:29:54:INFO:Updating latent states
2024-03-24 06:29:54:INFO:Updating model parameters
2024-03-24 06:30:04:INFO:Q value = -38071174.20013606
2024-03-24 06:30:04:INFO:Up

2024-03-24 06:40:01:INFO:Updating model parameters
2024-03-24 06:40:15:INFO:Q value = -38057943.95304071
2024-03-24 06:40:15:INFO:Updating latent states
2024-03-24 06:40:15:INFO:Updating model parameters
2024-03-24 06:40:20:INFO:Q value = -38057918.21881143
2024-03-24 06:40:20:INFO:Updating latent states
2024-03-24 06:40:20:INFO:Updating model parameters
2024-03-24 06:40:27:INFO:Q value = -38057887.40975611
2024-03-24 06:40:27:INFO:Updating latent states
Global seed set to 1


...

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-03-24 08:18:30:INFO:Updating latent states
2024-03-24 08:18:34:INFO:Updating model parameters
2024-03-24 08:18:40:INFO:Q value = nan
2024-03-24 08:18:40:INFO:Updating latent states
2024-03-24 08:18:41:INFO:Updating model parameters
2024-03-24 08:18:45:INFO:Q value = nan
2024-03-24 08:18:45:INFO:Updating latent states
2024-03-24 08:18:46:INFO:Updating model parameters
2024-03-24 08:18:49:INFO:Q value = nan
2024-03-24 08:18:49:INFO:Updating latent states
2024-03-24 08:18:49:INFO:Updating model parameters
2024-03-24 08:18:53:INFO:Q value = nan
2024-03-24 08:18:53:INFO:Updating latent states
2024-03-24 08:18:53:INFO:Updating model parameters
2024-03-24 08:18:56:INFO:Q value = nan
2024-03-24 08:18:56:INFO:Updating latent states
2024-03-24 08:18:57:INFO:Updat

2024-03-24 08:31:10:INFO:Updating model parameters
2024-03-24 08:31:15:INFO:Q value = -38055357.989219174
2024-03-24 08:31:15:INFO:Updating latent states
2024-03-24 08:31:15:INFO:Updating model parameters
2024-03-24 08:31:20:INFO:Q value = -38055352.47189325
2024-03-24 08:31:20:INFO:Updating latent states
2024-03-24 08:31:20:INFO:Updating model parameters
2024-03-24 08:31:24:INFO:Q value = -38055347.89342101
2024-03-24 08:31:24:INFO:Updating latent states
2024-03-24 08:31:24:INFO:Updating model parameters
2024-03-24 08:31:28:INFO:Q value = -38055345.17944645
2024-03-24 08:31:28:INFO:Updating latent states
2024-03-24 08:31:28:INFO:Updating model parameters
2024-03-24 08:31:39:INFO:Q value = -38055334.92716986
2024-03-24 08:31:39:INFO:Updating latent states
2024-03-24 08:31:39:INFO:Updating model parameters
2024-03-24 08:31:48:INFO:Q value = -38055327.309992254
2024-03-24 08:31:48:INFO:Updating latent states
2024-03-24 08:31:48:INFO:Updating model parameters
2024-03-24 08:31:52:INFO:Q va

2024-03-24 08:35:36:INFO:Updating latent states
2024-03-24 08:35:36:INFO:Updating model parameters
2024-03-24 08:35:40:INFO:Q value = -38055081.14541928
2024-03-24 08:35:40:INFO:Updating latent states
2024-03-24 08:35:40:INFO:Updating model parameters
2024-03-24 08:35:43:INFO:Q value = -38055075.44569803
2024-03-24 08:35:43:INFO:Updating latent states
2024-03-24 08:35:43:INFO:Updating model parameters
2024-03-24 08:35:47:INFO:Q value = -38055072.121968664
2024-03-24 08:35:47:INFO:Updating latent states
Global seed set to 8


8


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-03-24 08:35:56:INFO:Updating latent states
2024-03-24 08:35:58:INFO:Updating model parameters
2024-03-24 08:36:01:INFO:Q value = nan
2024-03-24 08:36:01:INFO:Updating latent states
2024-03-24 08:36:02:INFO:Updating model parameters
2024-03-24 08:36:05:INFO:Q value = nan
2024-03-24 08:36:05:INFO:Updating latent states
2024-03-24 08:36:06:INFO:Updating model parameters
2024-03-24 08:36:08:INFO:Q value = nan
2024-03-24 08:36:08:INFO:Updating latent states
2024-03-24 08:36:08:INFO:Updating model parameters
2024-03-24 08:36:10:INFO:Q value = nan
2024-03-24 08:36:10:INFO:Updating latent states
2024-03-24 08:36:11:INFO:Updating model parameters
2024-03-24 08:36:13:INFO:Q value = nan
2024-03-24 08:36:13:INFO:Updating latent states
2024-03-24 08:36:13:INFO:Updat